In [ ]:
import json
import os, sys
import csv
import requests
import time
from os import error

In [ ]:
# geojson file of locations and extents
file = open('/path/to/reference_data/middle_east_ports.geojsonl.json')
data = file.readlines()

# data variables
base_dir = "/path/to/ports_project/"

# customize template as needed, this creates request by month
template = open(base_dir+"template.json")
body = json.load(template)
months_duration = [{"jan":["01","01","31"]}, {"feb":["02","01","28"]},{"mar":["03","01","31"]},{"apr":["04","01","30"]},{"may":["05","01","31"]},{"jun":["06","01","30"]},{"jul":["07","01","31"]},{"aug":["08","01","31"]},{"sep":["09","01","30"]},{"oct":["10","01","31"]},{"nov":["11","01","30"]}]

# response from request with request id saved here
requests_file = open(base_dir+"traffic_analytics_requests/apac_ports_request_ids.txt","a")

# post variables for traffic analytics API, see documentation for requesting access token
url = "https://trafficanalytics.api.here.com/dailyAnalytics/requests"
access_token = ""
header_data = {'Authorization': 'Bearer ' + access_token}

# format coordinates for traffic analytics request
def format_coordinates(coords):
    formatted_coords = []
    coords_string = "POLYGON(("
    for c in coords:
        new_coords = str(c[0])+" "+str(c[1])
        formatted_coords.append(new_coords)
        coords_string = coords_string+new_coords+", "
        
    coords_string = coords_string[:-2]
    coords_string = coords_string+"))"
    return coords_string

# create and send requests to traffic analytics API
#   requests data for one month (January - November)
def create_requests(coords, file_name):
    body['queryFilter']['location'] = {"geometry" : coords}
    for month in months_duration:
        for k,v in month.items():
            county_responses_list = []
            month_str = k
            start = "2021-"+v[0]+"-"+v[1]
            end = "2021-"+v[0]+"-"+v[2]
            body['queryFilter']['startDate']= start
            body['queryFilter']['endDate']= end
            print(body)

# the Traffic Analytics API handles one request at a time. 
# Do not submit a request until the previous one is completed
# the requests are printed for copy and paste convenience

# DO NOT UNCOMMENT UNTIL API CAN HANDLE ASYNCH REQUESTS
#            r = requests.post(url, json=body, headers=header_data)
#             while True:
#                 time.sleep(20)
#                 try:
#             response = r.json()
#             request_id = response['requestId']
#             county_responses_list.append([month_str,file_name,request_id])
#             requests_file.write(json.dumps(county_responses_list))
#                     break
#                 except:
#                     print ("waiting ...")
    return
        

# create request from geojson file
for d in data:                   
    port = json.loads(d)
    port_name = port['properties']['NAME']
    port_coordinates = format_coordinates(port['geometry']['coordinates'][0])
    ta_request = create_requests(port_coordinates, port_name)
    port.clear()